In [1]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy

py_file_location = '../PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LABEL = 'trend'
MODEL = 'lgbc'

In [3]:
curated_data = pd.read_csv('../data/curated_data.csv')
curated_data = curated_data.drop(columns=['closePrice', 'closePriceNorm', 'log_ret_normalised_by_day', 'log_ret', 'stock'])
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [4]:
date = [datetime.datetime(year=2022, month=12, day = 31) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [5]:
function_map = ['min', 'max', 'std', 'mean', 'slope', 'skew', 'abs_diff_mean', 'diff_std', 'sum']

for i in [3, 5, 10, 20]:
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'log_ret_normalised_by_day_lag_{j}', f'closePriceNorm_lag_{j}', f'closePrice_lag_{j}'], axis = 1)
    
    for j in [3, 5, 10, 20]:
        if j > i:

            columns = []
            for col in ['log_ret', 'closePrice', 'closePriceNorm', 'log_ret_normalised_by_day']:
                for func in function_map:
                    columns.append(f'{col}_{func}_{j}')
            tmp_curated_data = tmp_curated_data.drop(columns=columns, axis = 1)

    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]

    ningxiang = NingXiang.NingXiang()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']

    ningxiang.read_in_train_data(train_x, train_y, val_x, val_y)

    ningxiang.set_model_type('Classification')

    ningxiang_output = ningxiang.get_rf_based_feature_combinations(1, 1, 4)

    ningxiang.export_ningxiang_output(f'../models/NingXiang_output_{LABEL}_lag_{i}')

    ningxiang.show_rf_stats()

    print('\n\n')

Max Lag: 3
NingXiang Initialised
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Successfully recorded model type: Classification
Begin fitting Random Forest
Finished fitting Random Forest
Successfully set object output address
{'closePrice_min_3': 0.006408426720203389, 'closePrice_max_3': 0.007130099288784341, 'closePrice_std_3': 0.01658025178447436, 'closePrice_mean_3': 0.005974958364082769, 'closePrice_slope_3': 0.01667990472410514, 'closePrice_skew_3': 0.014391488938821098, 'closePrice_abs_diff_mean_3': 0.012503534071260813, 'closePrice_diff_std_3': 0.012468634546493247, 'closePrice_sum_3': 0.0061984065739830755, 'log_ret_min_3': 0.01567448131584595, 'log_ret_max_3': 0.016516622009423186, 'log_ret_std_3': 0.019691902325902885, 'log_ret_mean_3': 0.010203277539324563, 'log_ret_slope_3': 0.024451232041358368, 'log_ret_skew_3': 0.02359981582156653, 'log_ret_abs_diff_mean_3': 0.017299930143376708, 'log_ret_diff_std_3': 0.018559908985135713, 'log_ret_sum_3

In [8]:
function_map = ['min', 'max', 'std', 'mean', 'slope', 'skew', 'abs_diff_mean', 'diff_std', 'sum']

for i in [3, 5, 10, 20]:
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'log_ret_normalised_by_day_lag_{j}', f'closePriceNorm_lag_{j}', f'closePrice_lag_{j}'], axis = 1)
    
    for j in [3, 5, 10, 20]:
        if j > i:

            columns = []
            for col in ['log_ret', 'closePrice', 'closePriceNorm', 'log_ret_normalised_by_day']:
                for func in function_map:
                    columns.append(f'{col}_{func}_{j}')
            tmp_curated_data = tmp_curated_data.drop(columns=columns, axis = 1)
    
    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]
    test = tmp_curated_data[(tmp_curated_data['date'] >= test_date_start)]

    jiaocheng = JiaoCheng.JiaoCheng()

    train_x = train.drop(columns=['date', 'trend'], axis = 1)
    train_y = train['trend']

    val_x = val.drop(columns=['date', 'trend'], axis = 1)
    val_y = val['trend']
    
    test_x = test.drop(columns=['date', 'trend'], axis = 1)
    test_y = test['trend']

    with open(f'../models/NingXiang_output_{LABEL}_lag_{i}.pickle', 'rb') as f:
        ningxiang_output = pickle.load(f)

    ningxiang_output = list(ningxiang_output.items())
    ningxiang_output = dict(ningxiang_output)

    jiaocheng = JiaoCheng.JiaoCheng()

    from lightgbm import LGBMClassifier as LGBC

    jiaocheng.read_in_model(LGBC, 'Classification')

    jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

    del train_x
    del train_y
    del val_x
    del val_y

    parameter_choices = {
        'n_estimators': (25, 50, 100, 200, 400),
        'max_depth': (3, 6, 12, 24, 48, 96, 192),
        'subsample': (0.4, 0.55, 0.7, 0.85),
        'colsample_bytree': (0.4, 0.55, 0.7, 0.85),
        'learning_rate': (0.00000001, 0.000001, 0.0001, 0.01, 0.1),
        'reg_alpha': (0.0001, 0.001, 0.01, 0.1, 1, 10)

    }

    jiaocheng.set_hyperparameters(parameter_choices)

    jiaocheng.set_non_tuneable_hyperparameters({'random_state': 19260817, 'n_jobs': -1})

    jiaocheng.set_features(ningxiang_output)

    jiaocheng.set_tuning_order(['features', 'max_depth', 'colsample_bytree', 'subsample', 'n_estimators', 'reg_alpha', 'learning_rate'])

    jiaocheng.set_hyperparameter_default_values({
        'features': 0,
        'n_estimators': 50,
        'max_depth': 12,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'learning_rate': 0.1,
        'reg_alpha': 0.01}
    )

    try:
        jiaocheng.read_in_tuning_result_df(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    except:
        pass

    jiaocheng.set_tuning_result_saving_address(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    jiaocheng.set_best_model_saving_address(f'../models/tmp_models/jiaocheng_{MODEL}_{LABEL}_lag_{i}.pickle')

    jiaocheng.tune()

    jiaocheng.view_best_combo_and_score()

    print('\n\n')

Max Lag: 3
JiaoCheng Initialised
JiaoCheng Initialised
Successfully read in model <class 'lightgbm.sklearn.LGBMClassifier'>, which is a Classification model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Successfully recorded tuneable feature combination choices and updated relevant internal structures
Successfully read in tuning result of 161 rows
Successfully set tuning output address
Successfully set best model output address

Default combo: [1, 2, 2, 2, 4, 2, 0] 


ROUND 1

Round 1 
Hyperparameter: features (index: 6) 

As new Best Combo (1, 2, 2, 2, 4, 2, 0) was read in, best_clf is set to None
Already Trained and Tested combination (1, 2, 2, 2, 4, 2, 0), which had val score of 0.5574
        Current best combo: (1, 2, 2, 2, 4, 2, 0) with val score 0.5574. 
        Has trained 161 of 940800 combinations so f

In [7]:
import numpy as np

pos = 0
test_y = np.array(test_y)
for i in range(len(test_y)):
    if test_y[i] == 1:
        pos += 1

print(pos/len(test_y))

0.492972972972973
